# Demo: Exploring the Iris Dataset

We'll start this tutorial with a demo to whet your appetite for learning more. This section purposely moves quickly through many of the concepts (e.g. data, marks, encodings, aggregation, data types, selections, etc.)
We will return to treat each of these in more depth later in the tutorial, so don't worry if it all seems to go a bit quickly!

In the live tutorial, this will be done from scratch in a blank notebook.
However, for the sake of people who want to look back on what we did live, I'll do my best to reproduce the examples and the discussion here.

## 1. Imports and Data

We'll start with importing the [Altair package](http://altair-viz.github.io/) and enabling the appropriate renderer (if necessary):

In [ ]:
import altair as alt

# Altair plots render by default in JupyterLab and nteract

# Uncomment/run this line to enable Altair in the classic notebook (not in JupyterLab)
# alt.renderers.enable('notebook')

# Uncomment/run this line to enable Altair in Colab
# alt.renderers.enable('colab')

Now we'll use the [vega_datasets package](https://github.com/altair-viz/vega_datasets), to load an example dataset:

In [ ]:
from vega_datasets import data

iris = data.iris()
iris.head()

Notice that this data is in columnar format: that is, each column contains an attribute of a data point, and each row contains a single instance of the data (here, a single make & model of car).

## 2. Zero, One, and Two-dimensional Charts

Using Altair, we can being to explore this data.

The most basic chart contains the dataset, along with a mark to represent each row:

In [ ]:
alt.Chart(iris).mark_point()

This is a pretty silly chart, because it consists of 150 points, all laid-out on top of each other. But this *is* technically a representation of the data!

To make it more interesting, we need to *encode* columns of the data into visual features of the plot (e.g. x-position, y-position, size, color, etc.)

Let's encode petal length on the x-axis using the ``encode()`` method:

In [ ]:
alt.Chart(iris).mark_point().encode(
    x='petalLength'
)

This is a bit better, but the ``point`` mark is probably not the best for a 1D chart like this.

Let's try the ``tick`` mark instead:

In [ ]:
alt.Chart(iris).mark_tick().encode(
    x='petalLength'
)

Another thing we might do with the points is to expand them into a 2D chart by also encoding the y value. We'll return to using ``point`` markers, and put ``petalWidth`` on the y-axis

In [ ]:
alt.Chart(iris).mark_point().encode(
    x='petalLength',
    y='petalWidth'
)

## 3 Simple Interactions

One of the nicest features of Altair is the grammar of interaction that it provides.
The simplest kind of interaction is the ability to pan and zoom along charts; Altair contains a shortcut to enable this via the ``interactive()`` method:

In [ ]:
alt.Chart(iris).mark_point().encode(
    x='petalLength',
    y='petalWidth'
).interactive()

This lets you click and drag, as well as use your computer's scroll/zoom behavior to zoom in and out on the chart.

We'll see other interactions later.

## 4. A Third Dimension: Color

A 2D plot allows us to encode two dimensions of the data. Let's look at using color to encode a third:

In [ ]:
alt.Chart(iris).mark_point().encode(
    x='petalLength',
    y='petalWidth',
    color='species'
)

Notice that when we use a categorical value for color, it chooses an appropriate color map for categorical data.

Let's see what happens when we use a continuous color value:

In [ ]:
alt.Chart(iris).mark_point().encode(
    x='petalLength',
    y='petalWidth',
    color='sepalLength'
)

A continuous color results in a color scale that is appropriate for continuous data.

This is one key feature of Altair: it chooses appropriate scales for the data type.

Similar, if we put the categorical value on the x or y axis, it will also create discrete scales:

In [ ]:
alt.Chart(iris).mark_tick().encode(
    x='petalLength',
    y='species',
    color='species'
)

Altair also can choose the orientation of the tick mark appropriately given the types of the x and y encodings:

In [ ]:
alt.Chart(iris).mark_tick().encode(
    y='petalLength',
    x='species',
    color='species'
)

## 5. Binning and aggregation

Let's return quickly to our 1D chart of petal length:

In [ ]:
alt.Chart(iris).mark_tick().encode(
    x='petalLength',
)

Another way we might represent this data is to creat a histogram: to bin the x data and show the count on the y axis.
In many plotting libraries this is done with a special method like ``hist()``. In Altair, such binning and aggregation is part of the declarative API.

To move beyond a simple field name, we use ``alt.X()`` for the x encoding, and we use ``'count()'`` for the y encoding:

In [ ]:
alt.Chart(iris).mark_bar().encode(
    x=alt.X('petalLength', bin=True),
    y='count()'
)

If we want more control over the bins, we can use ``alt.Bin`` to adjust bin parameters

In [ ]:
alt.Chart(iris).mark_bar().encode(
    x=alt.X('petalLength', bin=alt.Bin(maxbins=50)),
    y='count()'
)

If we apply another encoding (such as ``color``), the data will be automatically grouped within each bin:

In [ ]:
alt.Chart(iris).mark_bar().encode(
    x=alt.X('petalLength', bin=alt.Bin(maxbins=50)),
    y='count()',
    color='species'
)

If you prefer a separate plot for each category, the ``column`` encoding can help:

In [ ]:
alt.Chart(iris).mark_bar().encode(
    x=alt.X('petalLength', bin=alt.Bin(maxbins=50)),
    y='count()',
    color='species',
    column='species'
)

Binning and aggregation works in two dimensions as well; we can use the ``rect`` marker and visualize the count using the color:

In [ ]:
alt.Chart(iris).mark_rect().encode(
    x=alt.X('petalLength', bin=True),
    y=alt.Y('sepalLength', bin=True),
    color='count()'
)

Aggregations can be more than simple counts; we can also aggregate and compute the mean of a third quantity within each bin

In [ ]:
alt.Chart(iris).mark_rect().encode(
    x=alt.X('petalLength', bin=True),
    y=alt.Y('sepalLength', bin=True),
    color='mean(petalWidth)'
)

## 6. Stacking and Layering

Sometimes a multi-panel view is helpful in order to see relationships between points.
Altair provides the ``hconcat()`` and ``vconcat()`` functions, and the associated ``|`` and ``&`` operators to concatenate charts together (note this is different than the column-wise faceting we saw above, because each panel contains the *entire* dataset)

In [ ]:
chart1 = alt.Chart(iris).mark_point().encode(
    x='petalWidth',
    y='sepalWidth',
    color='species'
)

chart2 = alt.Chart(iris).mark_point().encode(
    x='sepalLength',
    y='sepalWidth',
    color='species'
)

chart1 | chart2

A useful trick for this sort of situation is to create a base chart, and concatenate two slightly different copies of it:

In [ ]:
base = alt.Chart(iris).mark_point().encode(
    y='sepalWidth',
    color='species'
)

base.encode(x='petalWidth') | base.encode(x='sepalLength')

Another type of compound chart is a layered chart, built using ``alt.layer()`` or equivalently the ``+`` operator.
For example, we could layer points on top of the binned counts from before:

In [ ]:
counts = alt.Chart(iris).mark_rect(opacity=0.3).encode(
    x=alt.X('petalLength', bin=True),
    y=alt.Y('sepalLength', bin=True),
    color='count()'
)

points = alt.Chart(iris).mark_point().encode(
    x='petalLength',
    y='sepalLength',
    color='species'
)

counts + points

We'll see a use of this kind of chart stacking in the interactivity section, below.

## 7. Interactivity: Selections

Let's return to our scatter plot, and take a look at the other types of interactivity that Altair offers:

In [ ]:
alt.Chart(iris).mark_point().encode(
    x='petalLength',
    y='petalWidth',
    color='species'
)

Recall that you can add ``interactive()`` to the end of a chart to enable the most basic interactive scales:

In [ ]:
alt.Chart(iris).mark_point().encode(
    x='petalLength',
    y='petalWidth',
    color='species'
).interactive()

Altair provides a general ``selection`` API for creating interactive plots; for example, here we create an interval selection:

In [ ]:
interval = alt.selection_interval()

alt.Chart(iris).mark_point().encode(
    x='petalLength',
    y='petalWidth',
    color='species'
).properties(
    selection=interval
)

Currently this selection doesn't actually do anything, but we can change that by conditioning the color on this selection:

In [ ]:
interval = alt.selection_interval()

alt.Chart(iris).mark_point().encode(
    x='petalLength',
    y='petalWidth',
    color=alt.condition(interval, 'species', alt.value('lightgray'))
).properties(
    selection=interval
)

The nice thing about this selection API is that it *automatically* applies across any compound charts; for example, here we can horizontally concatenate two charts, and since they both have the same selection they both respond appropriately:

In [ ]:
interval = alt.selection_interval()

base = alt.Chart(iris).mark_point().encode(
    y='petalWidth',
    color=alt.condition(interval, 'species', alt.value('lightgray'))
).properties(
    selection=interval
)

base.encode(x='petalLength') | base.encode(x='sepalLength')

The properties of the interval selection can be easily modified; for example, we can specify that it only applies to the x encoding:

In [ ]:
interval = alt.selection_interval(encodings=['x'])

base = alt.Chart(iris).mark_point().encode(
    y='petalWidth',
    color=alt.condition(interval, 'species', alt.value('lightgray'))
).properties(
    selection=interval
)

base.encode(x='petalLength') | base.encode(x='sepalLength')

We can do even more sophisticated things with selections as well.
For example, let's make a histogram of the number of records by species, and stack it on our scatter plot:

In [ ]:
interval = alt.selection_interval()

base = alt.Chart(iris).mark_point().encode(
    y='petalLength',
    color=alt.condition(interval, 'species', alt.value('lightgray'))
).properties(
    selection=interval
)

hist = alt.Chart(iris).mark_bar().encode(
    x='count()',
    y='species',
    color='species'
).properties(
    width=800,
    height=80
).transform_filter(
    interval
)

scatter = base.encode(x='petalWidth') | base.encode(x='sepalWidth')

scatter & hist

This demo has covered a number of the available components of Altair.
In the following sections, we'll look into each of these a bit more systematically.